In [30]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [31]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

In [32]:
X = X[(y == '1') | (y == '3') | (y == '5') | (y == '7')]
y = y[(y == '1') | (y == '3') | (y == '5') | (y == '7')]

map = {'1': 0, '3': 1, '5': 2, '7': 3}
y = y.map(map)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [34]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_torch  = torch.tensor(X_test,  dtype=torch.float32)
y_test_torch  = torch.tensor(y_test.to_numpy(),  dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset  = TensorDataset(X_test_torch,  y_test_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

In [35]:
n_qubits = 8
n_layers = 1

dev = qml.device('lightning.gpu', wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qlayer(inputs, weights):
    qml.AngleEmbedding(inputs, wires=list(np.arange(n_qubits)), rotation='X')
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return tuple(qml.expval(qml.PauliZ(i)) for i in range(n_qubits))

In [36]:
class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 128)
        self.fc2 = nn.Linear(128, n_qubits)
        self.scale = nn.Parameter(torch.tensor([2 * np.pi]))
        self.qnn_weights = nn.Parameter(torch.rand(n_layers, n_qubits, 3) * 1e-3)
        self.output_layer = nn.Linear(n_qubits, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(x) * self.scale
        batch_size = x.size(0)
        out = torch.empty((batch_size, n_qubits), dtype=torch.float32, device=x.device)
        for batch_idx in range(batch_size):
            expvals = qlayer(x[batch_idx], self.qnn_weights)
            out[batch_idx] = torch.stack(expvals)
        x = self.output_layer(out)
        return x

In [37]:
class ClassicalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 128)
        self.fc2 = nn.Linear(128, 8)
        self.output_layer = nn.Linear(8, 4)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output_layer(x)
        return x

In [38]:
sample_size_for_each_class_classical = 5
batch_size_classical = 32

X_train_sampled_classical = []
y_train_sampled_classical = []


for class_label in np.unique(y_train):
    class_indices = np.where(y_train == class_label)[0]
    sampled_indices = np.random.choice(class_indices, size=sample_size_for_each_class_classical, replace=False)

    X_train_sampled_classical.extend(X_train[sampled_indices])
    y_train_sampled_classical.extend(y_train.iloc[sampled_indices]) 

X_train_sampled_classical = np.array(X_train_sampled_classical)
y_train_sampled_classical = np.array(y_train_sampled_classical)

X_train_sampled_torch_classical = torch.tensor(X_train_sampled_classical, dtype=torch.float32)
y_train_sampled_torch_classical = torch.tensor(y_train_sampled_classical, dtype=torch.long)

train_sampled_dataset_classical = TensorDataset(X_train_sampled_torch_classical, y_train_sampled_torch_classical)
train_sampled_loader_classical = DataLoader(train_sampled_dataset_classical, batch_size=batch_size_classical, shuffle=True)

In [39]:
sample_size_for_each_class_quantum = 5
batch_size_quantum = 32

X_train_sampled_quantum = []
y_train_sampled_quantum = []


for class_label in np.unique(y_train):
    class_indices = np.where(y_train == class_label)[0]
    sampled_indices = np.random.choice(class_indices, size=sample_size_for_each_class_quantum, replace=False)

    X_train_sampled_quantum.extend(X_train[sampled_indices])
    y_train_sampled_quantum.extend(y_train.iloc[sampled_indices]) 

X_train_sampled_quantum = np.array(X_train_sampled_quantum)
y_train_sampled_quantum = np.array(y_train_sampled_quantum)

X_train_sampled_torch_quantum = torch.tensor(X_train_sampled_quantum, dtype=torch.float32)
y_train_sampled_torch_quantum = torch.tensor(y_train_sampled_quantum, dtype=torch.long)

train_sampled_dataset_quantum = TensorDataset(X_train_sampled_torch_quantum, y_train_sampled_torch_quantum)
train_sampled_loader_quantum = DataLoader(train_sampled_dataset_quantum, batch_size=batch_size_quantum, shuffle=True)

In [40]:
model_quantum = HybridClassifier()

learning_rate_quantum = 1e-3
batch_size_quantum = 32
epochs_quantum = 3

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def train_loop_quantum(dataloader, loss_fn, optimizer):
    model_quantum.train()
    model_quantum.to(torch_device)  
    for _, (X, y) in enumerate(dataloader):
        X, y = X.to(torch_device), y.to(torch_device)
        pred = model_quantum(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def test_loop_quantum(dataloader, loss_fn):
    model_quantum.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    model_quantum.to(torch_device)  
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(torch_device), y.to(torch_device)
            pred = model_quantum(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()

    print(f"Accuracy: {correct / (len(dataloader) * batch_size_quantum) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

    return test_loss

In [41]:
model_classical = ClassicalClassifier()

learning_rate_classical = 1e-3
batch_size_classical = 32
epochs_classical = 20

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def train_loop_classical(dataloader, loss_fn, optimizer):
    model_classical.train()
    model_classical.to(torch_device)  
    for _, (X, y) in enumerate(dataloader):
        X, y = X.to(torch_device), y.to(torch_device)
        pred = model_classical(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def test_loop_classical(dataloader, loss_fn):
    model_classical.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    model_classical.to(torch_device)  
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(torch_device), y.to(torch_device)
            pred = model_classical(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()

    print(f"Accuracy: {correct / (len(dataloader) * batch_size_classical) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

    return test_loss

In [42]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_classical.parameters(), lr=learning_rate_classical)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
start_time = time.time()

print(f"Before training sampled data for classical\n--------------------------")
for t in range(epochs_classical):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_classical(train_sampled_loader_classical, loss_fn, optimizer)
    test_loss = test_loop_classical(test_loader, loss_fn)
    scheduler.step(test_loss)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_classical(test_loader, loss_fn)

Before training sampled data for classical
--------------------------
Epoch 1
--------------------------
Accuracy: 22.014663696289062
Avg loss: 1.395014 

Epoch 2
--------------------------
Accuracy: 22.206703186035156
Avg loss: 1.373089 

Epoch 3
--------------------------
Accuracy: 22.74790382385254
Avg loss: 1.349440 

Epoch 4
--------------------------
Accuracy: 24.07472038269043
Avg loss: 1.324016 

Epoch 5
--------------------------
Accuracy: 26.204607009887695
Avg loss: 1.297568 

Epoch 6
--------------------------
Accuracy: 30.499300003051758
Avg loss: 1.270469 

Epoch 7
--------------------------
Accuracy: 38.32052993774414
Avg loss: 1.242235 

Epoch 8
--------------------------
Accuracy: 48.411312103271484
Avg loss: 1.213175 

Epoch 9
--------------------------
Accuracy: 59.96857452392578
Avg loss: 1.183759 

Epoch 10
--------------------------
Accuracy: 66.81214904785156
Avg loss: 1.154113 

Epoch 11
--------------------------
Accuracy: 70.14664459228516
Avg loss: 1.124366 


0.8823071035592915

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_classical.parameters(), lr=learning_rate_classical)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training for classical\n--------------------------")
for t in range(epochs_classical):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_classical(train_loader, loss_fn, optimizer)
    test_loss = test_loop_classical(test_loader, loss_fn)
    scheduler.step(test_loss)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_classical(test_loader, loss_fn)
print("Done!")

Before training for classical
--------------------------
Epoch 1
--------------------------
Accuracy: 97.76536560058594
Avg loss: 0.086503 

Epoch 2
--------------------------
Accuracy: 98.06214904785156
Avg loss: 0.086084 

Epoch 3
--------------------------
Accuracy: 98.30656433105469
Avg loss: 0.080448 

Epoch 4
--------------------------
Accuracy: 98.34147644042969
Avg loss: 0.090645 

Epoch 5
--------------------------
Accuracy: 98.62081146240234
Avg loss: 0.088699 

Epoch 6
--------------------------
Accuracy: 98.42876434326172
Avg loss: 0.122780 

Epoch 7
--------------------------
Accuracy: 98.49860382080078
Avg loss: 0.135819 

Epoch 8
--------------------------
Accuracy: 98.32402038574219
Avg loss: 0.155733 

Epoch 9
--------------------------
Accuracy: 98.39385223388672
Avg loss: 0.185733 

Epoch 10
--------------------------
Accuracy: 98.56843566894531
Avg loss: 0.174088 

Epoch 11
--------------------------
Accuracy: 98.62081146240234
Avg loss: 0.171816 

Epoch 12
--------

In [44]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_quantum.parameters(), lr=learning_rate_quantum)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
start_time = time.time()

print(f"Before training sampled data for quantum\n--------------------------")
for t in range(epochs_quantum):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_quantum(train_sampled_loader_quantum, loss_fn, optimizer)
    test_loss = test_loop_quantum(test_loader, loss_fn)
    scheduler.step(test_loss)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_quantum(test_loader, loss_fn)

Before training sampled data for quantum
--------------------------
Epoch 1
--------------------------
Accuracy: 29.975557327270508
Avg loss: 1.319976 

Epoch 2
--------------------------
Accuracy: 31.023042678833008
Avg loss: 1.280532 

Epoch 3
--------------------------
Accuracy: 31.70391082763672
Avg loss: 1.263052 

Training took 3.70 minutes.
Accuracy: 31.70391082763672
Avg loss: 1.263052 



1.263051777578599

In [45]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_quantum.parameters(), lr=learning_rate_quantum)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training for quantum\n--------------------------")
for t in range(epochs_quantum):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_quantum(train_loader, loss_fn, optimizer)
    test_loss = test_loop_quantum(test_loader, loss_fn)
    scheduler.step(test_loss)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_quantum(test_loader, loss_fn)

Before training for quantum
--------------------------
Epoch 1
--------------------------
Accuracy: 94.32611083984375
Avg loss: 0.247705 

Epoch 2
--------------------------
Accuracy: 96.45600128173828
Avg loss: 0.153005 

Epoch 3
--------------------------
Accuracy: 97.03211975097656
Avg loss: 0.129944 

Training took 41.44 minutes.
Accuracy: 97.03211975097656
Avg loss: 0.129944 



0.1299443712364362